In [1]:
#Importar as bibliotecas de manipulação de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Importar as planilhas formato csv
df1= pd.read_csv('train_test.csv') #Somente treinamento
df2= pd.read_csv('valid_xboost.csv') #Somente validação 

In [3]:
#Dividir o conjuto de dados de treinamento e escolher quais váriaveis não vão entrar
from sklearn.model_selection import train_test_split
X = df1.drop(['fid', 'lat', 'long', 'talhao', 'wkt_geom','Altitude1', 'produtivid' ], axis= 1)
y= df1['produtivid']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3) #70% treino e 30% teste

In [4]:
#Importar algoritmo
import xgboost as xgb
model = xgb.XGBRegressor() #criar um objeto recebendo o algoritmo
model.fit(X_train, y_train) #treinar o algoritmo

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [16]:
#Predição dentro dos dados de treino
y_pred = model.predict(X_test)

In [17]:
#Métricas de erro
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r_squared = r2_score(y_test, y_pred)
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r_squared)

MAE: 5.194486137629855
RMSE: 7.258134666004934
R²: 0.8488799217489591


In [8]:
#Predição na validação
y_prod_pred = model.predict(df2.drop(['produtivid','fid', 'lat', 'long', 'talhao', 'wkt_geom','Altitude1'], axis = 1))

In [12]:
#Métricas de erro
mae_p = mean_absolute_error(df2['produtivid'], y_prod_pred)
rmse_p = np.sqrt(mean_squared_error(df2['produtivid'], y_prod_pred))
r_squared_p = r2_score(df2['produtivid'], y_prod_pred)
print("MAE:", mae_p)
print("RMSE:", rmse_p)
print("R²:", r_squared_p)

MAE: 9.876386475279451
RMSE: 12.75980005222303
R²: 0.6675358465542578


In [13]:
from sklearn.model_selection import GridSearchCV

# Definir os hiperparâmetros a serem ajustados
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Criar XGBoost
xgb_model = xgb.XGBRegressor()

# Usar GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Imprimir os melhores hiperparâmetros encontrados
print("Melhores hiperparâmetros encontrados:")
print(grid_search.best_params_)

# Fazer predição com o melhor modelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Métricas de desempenho do modelo
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r_squared = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r_squared)

Melhores hiperparâmetros encontrados:
{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.9}
Mean Absolute Error: 4.993681325268513
Root Mean Squared Error: 6.9320945036119515
R-squared: 0.8621518075574309


In [14]:
#Predição na validação com melhores hiperparametros
pred_y = best_model.predict(df2.drop(['produtivid','fid', 'lat', 'long', 'talhao', 'wkt_geom','Altitude1'], axis = 1))

In [15]:
#Métricas de erro
p_mae = mean_absolute_error(df2['produtivid'], pred_y)
p_rmse = np.sqrt(mean_squared_error(df2['produtivid'], pred_y))
r_squared = r2_score(df2['produtivid'],pred_y)
print("MAE:", mae_p)
print("RMSE:", rmse_p)
print("R²:", r_squared_p)

MAE: 9.876386475279451
RMSE: 12.75980005222303
R²: 0.6675358465542578
